In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
from functions_for_parameter_exploration import *
import time

# Set file locations
advanced_settings_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTanjc08kc5vIN-icUMzMEGA9bJuDesLX8V_u2Ab6zSC4MOhLZ8Jrr18DL9o4ofKIrSq6FsJXhPWu3F/pub?gid=0&single=true&output=csv"

# Read in emissions from other sectors
sector_emissions_dict = set_sector_emissions_dict()
# Read in the advanced settings from the google sheet
params = read_advanced_settings(advanced_settings_url)
# Set the datablock
datablock_init = datablock_setup()

# Set the scenario parameters
set_baseline_scenario(params);

starttime = time.time()
SSR_result, emissions_result = SSR_emissions(datablock_init, params, sector_emissions_dict)
lasttime = time.time()
duration = lasttime - starttime

print(f"SSR = {SSR_result:.8f}; GHG = {emissions_result:.8f}")
# This is my checksum for debugging: 
# SSR = 0.67366432; GHG = 94.22954995

# Reproduce the 2d plot JP made
n1 = 3
n2 = 2
name1 = "horticulture"
x1_min = 0
x1_max = 200
name2 = "foresting_pasture"
x2_min = 15
x2_max = 25

# Now make the arrays
x1_vals = np.linspace(x1_min, x1_max, n1)
x2_vals = np.linspace(x2_min, x2_max, n2)
SSR_results = np.empty((n1,n2))
emissions_results = np.empty((n1,n2))
x1_mat = np.empty((n1,n2))
x2_mat = np.empty((n1,n2))

starttime = time.time()
for i1, x1 in enumerate(x1_vals):
    params[name1] = x1

    for i2, x2 in enumerate(x2_vals):
        params[name2] = x2

        SSR_results[i1, i2], emissions_results[i1, i2] = SSR_emissions(datablock_init, params, sector_emissions_dict)
        x1_mat[i1, i2] = x1
        x2_mat[i1, i2] = x2

    lasttime = time.time()
    duration = lasttime - starttime
    eta = (duration / (i1+1) * (n1-i1-1))/60
    print(f"Done i1 = {i1:2} of {n1:2}; eta = {eta:.2f} minutes")

totaltime = time.time()
time_per_function_call = (totaltime - starttime) / (n1 * n2)
print(f"time_per_function_call = {time_per_function_call:.2f}")

#plt.imshow(SSR_results, cmap='viridis', interpolation='nearest', extent=[x2_min, x2_max, x1_min, x1_max], aspect='auto')
plt.imshow(x1_mat, cmap='viridis', interpolation='nearest', extent=[x2_min, x2_max, x1_min, x1_max], origin='lower', aspect='auto')
plt.colorbar(label='SSR')
plt.ylabel(name1)
plt.xlabel(name2)


plt.imshow(emissions_results, cmap='rainbow', interpolation='nearest', extent=[x2_min, x2_max, x1_min, x1_max], aspect='auto')
plt.colorbar(label='Emissions')
plt.ylabel(name1)
plt.xlabel(name2)
